# 土地价格预测

## 导入所需要的包

In [1]:
#导⼊相关包
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

# 拆分预测数据和训练数据
from sklearn.model_selection import train_test_split

#聚类
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 模型
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingClassifier
from mlxtend.regressor import StackingCVRegressor

#随机寻参
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

#交叉验证
from sklearn.model_selection import KFold 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit


#模型评估
from sklearn import metrics

In [2]:
# 绘图
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

import seaborn as sns
sns.set(style='white',context='notebook',palette='muted',font='SimHei')  # 解决Seaborn中文显示问题并调整字体大小) # 设置sns样式

# 可视化树形图
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
from io import StringIO

# 预测情况绘制地图
import geopandas
import shapefile  # 使用pyshp
import numpy as np
from random import random
from pathlib import Path
import os
import matplotlib as mpl
%matplotlib inline


from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

from scipy import stats
from scipy.stats import norm, skew #for some statistics

# 一、数据分析

## 1.1 导入数据

有 1844 个训练数据实例。属性总数等于 46，其中 45 个是定量属性。

In [3]:

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib


#导⼊相关包
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

# 拆分预测数据和训练数据
from sklearn.model_selection import train_test_split

#聚类
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 模型
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingClassifier
from mlxtend.regressor import StackingCVRegressor

#随机寻参
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform

#交叉验证
from sklearn.model_selection import KFold 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit


#模型评估
from sklearn import metrics

def train_model():
    train=pd.read_csv('./data/建设用地处理后数据.csv')
    data=pd.read_csv('./data/建设用地原始数据.csv')
    X = train.drop(['样点地价'], axis = 1)
    y = train['样点地价']

    random_state = 42 # 随机数

    n_jobs = -1 # 工作线程 

    cv = KFold(n_splits=5)   # 交叉验证
    # 拆分为训练和测试数据 
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)
    # 使用 stacking 堆栈模型

    reg_1 = RandomForestRegressor()             
    reg_2 = XGBRegressor()
    reg_3 = LGBMRegressor()
    reg_4 = GradientBoostingRegressor()
    reg_5 = ExtraTreesRegressor()

    models = [
        (reg_1),
        (reg_2),
        (reg_3),
        (reg_4),
        (reg_5),
        ]
        

    # 创建stacking模型
    reg = StackingCVRegressor(regressors = models, meta_regressor = reg_4 ,cv = cv, verbose = 0 , random_state = random_state , n_jobs = n_jobs)

    reg.fit(X_train, y_train)
    # Predicting Test data with the model
    y_test_pred = reg.predict(X_test)#输入预测集获得预测值

    accuracy = metrics.r2_score(y_test, y_test_pred)
    joblib.dump(reg, '建设用地.model')
    print('Model Training Finished.\n\tAccuracy obtained: {}'.format(accuracy))


#### 部署

In [5]:
import os
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
import joblib
app = Flask(__name__)
api = Api(app)

if not os.path.isfile('建设用地.model'):
    train_model()

model = joblib.load('建设用地.model')

class MakePrediction(Resource):
   @staticmethod
   def post():
       posted_data = request.get_json()
       sepal_length = posted_data['sepal_length']
       sepal_width = posted_data['sepal_width']
       petal_length = posted_data['petal_length']
       petal_width = posted_data['petal_width']

       prediction = model.predict([[sepal_length, sepal_width, petal_length, petal_width]])[0]
       if prediction == 0:
           predicted_class = 'Iris-setosa'
       elif prediction == 1:
           predicted_class = 'Iris-versicolor'
       else:
           predicted_class = 'Iris-virginica'

       return jsonify({
           'Prediction': predicted_class
       })

api.add_resource(MakePrediction, '/predict')

from gevent import pywsgi
 
if __name__ == '__main__':
    from wsgiref.simple_server import make_server
    server = make_server('127.0.0.1',5000,app)
    server.serve_forever()
    app.run()

KeyboardInterrupt: 